In [1]:
import os
import json

import gdal
from osgeo import ogr
from osgeo import osr

# Sample code to extract WGS84 bounds from Geotif files

In [4]:
def get_wgs84_bounds(fname):
    src = gdal.Open(fname)
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
    lrx = ulx + (src.RasterXSize * xres)
    lry = uly + (src.RasterYSize * yres)
    
    # Setup the source projection - you can also import from epsg, proj4...
    source = osr.SpatialReference()
    source.ImportFromWkt(src.GetProjection())

    # The target projection
    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)

    # Create the transform from source to WGS84
    transform = osr.CoordinateTransformation(source, target)

    # Transform the points
    # top-left corner
    lng1, lat1, z1 = transform.TransformPoint(ulx, uly)
    # top-right corner
    lng2,lat2,z2 = transform.TransformPoint(lrx, uly)
    # bot-right corner
    lng3, lat3, z2 = transform.TransformPoint(lrx, lry)
    #bot-left corner
    lng4, lat4, z2 = transform.TransformPoint(ulx, lry)
    
    
    return [lng1,lat1,lng2,lat2,lng3,lat3,lng4,lat4]

### Original tif files not supplied, but can be downloaded via link in Readme

### For each map, the WGS84 bounds were already extracted and saved in json dir.

In [6]:
data_dir = 'dataset/TOP50raster'
json_dir = os.path.join(data_dir,'json')
tif_dir = os.path.join(data_dir,'tif')

for fname in os.listdir(tif_dir):
    if '.tif' in fname:
        map_fname = os.path.join(tif_dir,fname)
        
        json_fname = os.path.join(json_dir,fname.replace('.tif','.json'))
        bounds = get_wgs84_bounds(map_fname)
        obj = {'label_coordinates_WGS84': bounds}
        json.dump(obj,open(json_fname,'w'))
    